In [11]:
import requests
from bs4 import BeautifulSoup

url = 'https://www.serebii.net/card/paradoxrift/'
soup = BeautifulSoup(requests.get(url).text, 'html.parser')


In [20]:
# Pretty print the HTML
print(soup.prettify())


<!DOCTYPE html>
<head>
 <title>
  Paradox Rift - Serebii.net Pokémon Card Database
 </title>
 <link crossorigin="" href="https://securepubads.g.doubleclick.net/" rel="preconnect"/>
 <link crossorigin="" href="https://cdn.consentmanager.mgr.consensu.org/" rel="preconnect"/>
 <link crossorigin="" href="https://kumo.network-n.com/" rel="preconnect"/>
 <meta content="no" http-equiv="imagetoolbar"/>
 <link href="/style/layout.css" rel="stylesheet" type="text/css">
  <meta content="width=device-width,initial-scale=1" id="viewport-meta" name="viewport"/>
  <!-- GDPR Compliancy -->
  <style>
   body {
	--cmpBgColor: #507C36;
	--cmpTextColor: #000000;
	--cmpLinkColor: #ffffff;
	--cmpPurposesColor: #000000;
	--cmpBrandColor: #000000;
	--cmpLogo: url('https://www.serebii.net/extralogo.png');
}
  </style>
  <!-- GDPR Compliancy -->
  <link href="/serebii-opensearch.xml" rel="search" title="Serebii Open Search" type="application/opensearchdescription+xml">
   <link href="/favicon.ico" rel="shortcut

In [12]:
# Make a request to the robots.txt file
response = requests.get('https://www.serebii.net/robots.txt')

# Print the content of the response
print(response.text)


User-agent: *
Disallow: /hidden/ranch/
Disallow: /crossword/


In [13]:
# Find table rows, detailed by <tr> tag
rows = soup.find_all('tr')

In [42]:
import pandas as pd

paradox_rift = pd.DataFrame(columns=['Collection','Card Name','Card Number','Card Image URL', 'Card URL'])

for row in rows:
    cells = row.find_all('td')
    a_tag = cells[0].find('a')
    if a_tag is not None:
        collection_name = a_tag.text.strip()
        card_name_tag = cells[2].find('font')
        card_number_tag = cells[0].find('br').next_sibling
        url_tag = cells[0].find('a')
        if card_name_tag is not None:
            card_name = card_name_tag.text.strip()
            card_image_url = cells[1].find('img')['src']
            card_number = card_number_tag.text.strip()
            # Url is after second href
            
            print(f"This card is from: {collection_name} ", f"The card name is: {card_name} ", f"The card number is: {card_number} ", f"The picture url is {card_image_url} ", f"The card url is: {url}")
        else:
            continue
    else:
        continue

This card is from: Paradox Rift  The card name is: Surskit  The card number is: 1 / 182  The picture url is /card/th/paradoxrift/1.jpg  The card url is: /card/paradoxrift/
This card is from: Paradox Rift  The card name is: Masquerain  The card number is: 2 / 182  The picture url is /card/th/paradoxrift/2.jpg  The card url is: /card/paradoxrift/
This card is from: Paradox Rift  The card name is: Froslass  The card number is: 3 / 182  The picture url is /card/th/paradoxrift/3.jpg  The card url is: /card/paradoxrift/
This card is from: Paradox Rift  The card name is: Pansage  The card number is: 4 / 182  The picture url is /card/th/paradoxrift/4.jpg  The card url is: /card/paradoxrift/
This card is from: Paradox Rift  The card name is: Simisage  The card number is: 5 / 182  The picture url is /card/th/paradoxrift/5.jpg  The card url is: /card/paradoxrift/
This card is from: Paradox Rift  The card name is: Dwebble  The card number is: 6 / 182  The picture url is /card/th/paradoxrift/6.jpg 

C:\Users\tesch\AppData\Local\Temp\ipykernel_19940\3806544029.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
